### Introduction

Welcome to Priceless Wheels! In this project, our goal is to build a model that can accurately predict the price of a used vehicle based on various factors such as make, model, year, mileage, and condition. The automobile industry is one of the largest and most competitive industries in the world, with millions of vehicles being sold each year. The price of a vehicle can have a significant impact on a consumer's purchasing decision and it is important for both buyers and sellers to have an understanding of the market value of a vehicle. By using machine learning algorithms and data analysis, we aim to provide a reliable and robust model that can assist in determining the fair market value of a vehicle. Join us on this exciting journey as we delve into the world of vehicle price prediction.

### About the data

This data is scrapped from https://www.cardekho.com/. This data is meant for research and academic purposes only and is **not meant for commercial use**. This dataset contains about 38000 (thirty eight thousand) used cars listed in CarDekho in India. Download the data from https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv.

---

#### Importing the data and libraries

Let's start by importing the necessary libraries and the data.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import regex as re # for regex matching
import matplotlib.pyplot as plt # 
import seaborn as sns # seaborn for nice looking plots
sns.set() # setting seaborn default for plots

# removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/used-cars-dataset-cardekho/cars_details.csv
/kaggle/input/used-cars-dataset-cardekho/cars_overview.csv
/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv
/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv


Next, we will import the data and preview the first 5 rows

- Note: The ```usedCarSkuId``` is a unique identifier for each car. We would be using this column as the index of the dataframe
- Note: Since we are using github lfs for storing the CSV files, the link mentioned in the notebook will expire in some time. Please visit this [link](https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv) to get the data

In [2]:
# The data can be found on kaggle link: https://www.kaggle.com/datasets/sukritchatterjee/used-cars-dataset-cardekho

file_path = "/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv"
df = pd.read_csv(file_path, index_col="usedCarSkuId")

# sanity check
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (25,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,position,loc,myear,bt,tt,ft,km,ip,pi,images,...,owner_type,price_segment_new,template_name_new,page_template,template_Type_new,experiment,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,1,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,2,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,3,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,4,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,NaN,[{'img': ''}],...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,5,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,10+lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,12.0+-.03,NaN,NaN


The data has imported with some mixed data types. We will need to convert the data types as we explore the data.

In [3]:
# checking the shape of the dataset
shape = df.shape

print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 37814 rows and 139 columns


As we can see, there are 37,814 rows and 139 columns in the dataset.

Let's look at the data types of the columns.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37814 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Columns: 139 entries, position to Ground Clearance Unladen
dtypes: bool(3), float64(10), int64(28), object(98)
memory usage: 39.6+ MB


Let's take a look at the feature dictionary provided with the dataset.

In [5]:
feature_dictionary = pd.read_csv("/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv")
feature_dictionary

,Feature,Type,Sample,Missing Values,Unique Values,Description
0,position,int64,1,0,20,Position of the car in the list
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
...,...,...,...,...,...,...
135,experiment,object,control,0,1,control
136,Fuel Suppy System,object,NaN,5502,99,"Type of fuel supply system (Carburetor, Fuel I..."
137,Compression Ratio,object,NaN,27642,100,Compression ratio of the engine
138,Alloy Wheel Size,object,NaN,13146,18,Size of the alloy wheels in inches


Now comes the difficult task. Go through the feature dictionary and remove the unwanted columns - features which might be repeated but not totally apparent. Note down the columns which makes sense to keep and are not repeating and delete the rest.

In [6]:
columns_to_keep = [
    "loc",
    "myear",
    "bt",
    "tt",
    "ft"
    "km",
    "ip",
    "images",
    "imgCount",
    "threesixty",
    "dvn",
    "oem",
    "model",
    "variantName",
    "city_x",
    "pu",
    "discountValue",
    "utype",
    "carType", 
    "top_features",
    "comfort_features",
    "interior_features",
    "exterior_features",
    "safety_features",
    "Color",
    "Engine Type",
    "Max Power",
    "Max Torque",
    "No of Cylinder",
    "Values per Cylinder",
    "Value Configuration",
    "BoreX Stroke",
    "Turbo Charger",
    "Super Charger",
    "Length",
    "Width",
    "Height",
    "Wheel Base",
    "Front Tread",
    "Rear Tread",
    "Kerb Weight",
    "Gross Weight",
    "Gear Box",
    "Drive Type",
    "Seating Capacity",
    "Steering Type",
    "Turning Radius",
    "Front Brake Type",
    "Rear Brake Type",
    "Top Speed",
    "Acceleration",
    "Tyre Type",
    "No Door Numbers",
    "Cargo Volumn",
    "model_type_new",
    "state",
    "owner_type",
    "exterior_color",
    "Fuel Suppy System",
    "Compression Ratio",
    "Alloy Wheel Size",
    "Ground Clearance Unladen",
]

df.drop([x for x in df.columns if x not in columns_to_keep], axis=1, inplace=True)
print(f"After dropping some unnecessary columns, the dataset now has {df.shape[1]} columns. These columns are hand picked and will be be further analyzed.")

After dropping some unnecessary columns, the dataset now has 60 columns. These columns are hand picked and will be be further analyzed.


---

### Data cleaning and wrangling

* Dropping duplicate rows
* Fixing the values and data types of the columns
* Checking for multicollinearity and determining how to handle it
* Dropping irrelevant columns for the model
* Saving the cleaned data for the next part of the project

---

### Duplicates

We know that there should be no duplicates in the data. They are checked for and removed at the time of data collection. *(Trust me, I collected it myself 😉 )*

In [7]:
# checking for duplicate rows
duplucate_rows = df.duplicated().sum()

print(f"The number of duplicate rows are {duplucate_rows}.")

The number of duplicate rows are 1.


Somehow, one duplicate row sneaked in (that's awkward 🫣)! Let's check the rows.

In [8]:
# setting the duplicated index
duplicate_index = df.duplicated(keep=False)

# calling the duplicated index in a dataframe
df.loc[duplicate_index, :].sort_index()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
aa39e640-6183-4379-a517-9f5b2458b2a5,NaN,2014,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Swift VDI BSIV,Maruti,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN
ce8b30d8-c438-4a2f-bce1-5c1887a95495,NaN,2014,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Swift VDI BSIV,Maruti,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN


As we can see, the rows are exactly the same. This means we can safely drop the row. Since the we indexed by the usedCarSkuId, we need to drop one of them by their index.

In [9]:
# making a copy with the duplicated rows dropped
df2 = df.drop_duplicates().copy()

# checking for duplicate rows in the new dataframe
dup = df2.duplicated().sum()

print(f"The number of duplicate rows are {dup}.")

The number of duplicate rows are 0.


In [10]:
# sanity check
df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,Gomti Nagar,2016,Hatchback,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Maruti Wagon R LXI CNG,Maruti,...,5.000,180-liters,used,uttar pradesh,Silver,first,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,Borivali West,2015,Hatchback,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Maruti Celerio Green VXI,Maruti,...,5.000,235-litres,used,maharashtra,Grey,first,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,JASOLA,2015,Sedan,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Honda Amaze S Plus i-VTEC,Honda,...,4.000,400-litres,used,delhi,Silver,second,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Wagon R LXI CNG,Maruti,...,4.000,NaN,used,delhi,Silver,second,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,MUV,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,Maruti Ertiga VXI CNG,Maruti,...,5.000,NaN,used,maharashtra,White,first,NaN,12.0+-.03,NaN,NaN


Great! Now that we have removed the duplicate rows, let's look at missing values

---

### Fixing the data types

Let's look at the data types of the columns and in the process we can check if we need to clean any them to make them more useful, such as converting them to a numeric type.

*When we do feature engineering we will further modify these columns and might also drop a few of them*

We already know about the current data types of the features from the ```feature_dictionary```.

In [11]:
# Filter the dataframe to only include the columns_to_keep
new_feature_dict = feature_dictionary.copy()

# Drop the rows which Feature is not in the columns_to_keep list
row_index = [i for i, x in enumerate(new_feature_dict['Feature']) if x not in columns_to_keep]
new_feature_dict.drop(row_index, inplace=True)

new_feature_dict

,Feature,Type,Sample,Missing Values,Unique Values,Description
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
7,ip,int64,0,0,2,Insurance premium of the car
9,images,object,[{'img': 'https://images10.gaadi.com/usedcar_i...,0,37135,Number of images of the car
10,imgCount,int64,15,0,54,Count of images of the car
11,threesixty,bool,False,0,2,Whether 360 degree view of car is available or...
12,dvn,object,Maruti Wagon R LXI CNG,0,4159,Dealer verification status of the car
13,oem,object,Maruti,0,46,Original equipment manufacturer of the car


We know that if a column has the ```object``` type, it means it could have mixed types as well as strings. Let's try to convert them to a numerical type wherver we encouter something like Length or '12mm'.

In [12]:
# A helper function to print all the types in a column for a dataframe
def print_samples_from_every_type(df: pd.DataFrame, col: str):
    # Get all the types in the column.
    types = df[col].apply(type).unique()
    for t in types:
      # Print 10 samples from each type.
        print(f"Samples for type {t}: ")
        print(df[df[col].apply(type) == t].sample(10)[col])
        print()

In [13]:
# Make a copy of the original dataframe so that we can roll back to it anytime.
cars_df = df2.copy()

---

Let's list down all the columns which need the standard string treatment: 
1. Convert column to be a string type
2. Strip the whitespace 
3. Make it lowercase

*Here we are also give some of the other columns (that we wish to convert later) the standard treatment (it will help us later)*

In [14]:
columns_str_to_lower = [
    'loc',
    'bt',
    'tt',
    'images',
    'dvn',
    'oem',
    'model',
    'variantName',
    'city_x',
    # 'pu': 3,70,000 -> 370000
    'utype',
    'carType',
    'top_features',
    'comfort_features',
    'interior_features',
    'exterior_features',
    'safety_features',
    'Color',
    'Engine Type',
    'Max Power', # Max Power : 33.54bhp@4000 rpm -> 2 columns 33.54, 4000
    'Max Torque', # Max Torque : 40.2Nm@3500 rpm -> 2 columns 40.2, 3500
    # No of Cylinder : float -> int,
    # Values per Cylinder : float -> int, !! Fix the name -> Valve per Cylinder
    'Value Configuration', # !! Fix the name -> Valve Configuration
    'BoreX Stroke', # BoreX Stroke : 69 x 72 mm -> Bore: 69, Stroke: 72
    'Turbo Charger', # Convert to boolean
    'Super Charger', # Convert to boolean
    'Length', # Length : 3599mm -> 3599
    'Width', # Width : 1495mm -> 1495
    'Height', # Height : 1700mm -> 1700
    'Wheel Base', # Wheel Base : 2400mm -> 2400
    'Front Tread', # Front Tread : 1295mm -> 1295
    'Rear Tread', # Rear Tread : 1295mm -> 1295
    'Kerb Weight', # Kerb Weight : 960kg -> 960
    'Gross Weight', # Gross Weight : 1350kg -> 1350
    'Gear Box', # might need some additional cleaning
    'Drive Type', # might need some additional cleaning
    # Seating Capacity : float -> int
    'Steering Type', # might need some additional cleaning
    'Turning Radius', # Turning Radius : 4.6 metres -> 4.6
    'Front Brake Type', # might need some additional cleaning
    'Rear Brake Type', # might need some additional cleaning
    'Top Speed', # Top Speed : 137 kmph -> 137
    'Acceleration', # Acceleration : 13.5 seconds -> 13.5
    'Tyre Type', # might need some additional cleaning
    # No Door Numbers : float -> int !! fix name
    'Cargo Volumn', # 'Cargo Volumn' : 300 litres -> 300, !! fix name
    'model_type_new', # !! fix name
    'state',
    'exterior_color',
    'owner_type', # might need some additional cleaning
    'Fuel Suppy System', # might need some additional cleaning
    # Compression Ratio : 10.0:1 -> 10.0
    'Alloy Wheel Size', # Alloy Wheel Size : convert to float
    'Ground Clearance Unladen', # Ground Clearance Unladen : 170mm -> 170
]

In [15]:
# Apply the changes
temp_arr = []

for col in columns_str_to_lower:
    cars_df[col] = cars_df[col].astype(str).str.strip().str.lower()
    
    # Print a nice sample
    while (True):
        found = 0
        vals = [v for v in cars_df[col].sample(100)]
        for val in vals:
            if val is not None:
                found = 1
                vals = val
                break
        
        if found == 1:
            temp_arr.append({ 
                "Feature" : col, 
                "Type": f'{cars_df[col].dtype}', 
                "Unique": cars_df[col].nunique(), 
                "Sample": vals
            })
            break

pd.DataFrame(temp_arr)

,Feature,Type,Unique,Sample
0,loc,object,398,gomtinagar vistar
1,bt,object,12,hatchback
2,tt,object,2,manual
3,images,object,37135,[{'img': 'https://images10.gaadi.com/usedcar_i...
4,dvn,object,4128,hyundai i20 asta opt diesel
5,oem,object,46,land rover
6,model,object,382,maruti swift
7,variantName,object,3430,i vtec v
8,city_x,object,617,bangalore
9,utype,object,2,dealer


There is an additional issues here that needs to be addressed: 
Some of the columns have **more unique values** than are expected. For eg: `Turbo Charger` should either be a yes or a no or a NaN, beter are `5` unique values. The following are the columns suspected to have a higher than expected cardinality due to some quality issues: 
1. `Value Configuration`
2. `Turbo Charger`
3. `Gear Box`
4. `Drive Type`
5. `Steering Type`
6. `Front Brake Type`
7. `Rear Brake Type`
8. `Tyre Type`


In [16]:
# Make a list of such columns which seems to have a higher than expected cardinality
columns_unexpected_cardinality = [
    'Value Configuration',
    'Turbo Charger',
    'Gear Box',
    'Drive Type',
    'Steering Type',
    'Front Brake Type',
    'Rear Brake Type',
    'Tyre Type',
]

---

##### `Value Configuration`

In [17]:
cars_df[columns_unexpected_cardinality[0]].value_counts()

dohc                    23502
nan                      7831
sohc                     6200
undefined                 121
idsi                       66
dohc with vis              31
dohc with vgt              20
16 modules 48 cells        11
16-valve dohc layout        9
dohc with tis               9
vtec                        6
mpfi                        4
ohv / pushrod               3
Name: Value Configuration, dtype: int64

We have identified the following issues:
1. As we can see, the majority of it is `dohc`. There are a couple of variants of `dohc` like `dohc vis` but they are so few that it would be practical to replace all the columns containing any variant `dohc` to simply `dohc`
2. There are some `undefined` values which should be `NaN`
3. `mpfi` is a fuel injection system, so replace it with `NaN`
4. `vtec` engines can be `dohc` and `sohc` as well (`vtec` is an engine type developed by Honda in the 1960s). It would be best to replace this type by `NaN`, postpone handling it until null values handling (and there are only `6` rows, so it should be fine)

In [18]:
# Replace all the variants of `dohc` to simply `dohc`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vis', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vgt', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('16-valve dohc layout', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with tis', 'dohc')

# Replace `undefined`, `mpfi`, `vtec` with `NaN`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('undefined', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('mpfi', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('vtec', 'nan')

cars_df['Value Configuration'].value_counts()

dohc                   23571
nan                     7962
sohc                    6200
idsi                      66
16 modules 48 cells       11
ohv / pushrod              3
Name: Value Configuration, dtype: int64

---

##### `Turbo Charger`

In [19]:
cars_df[columns_unexpected_cardinality[1]].value_counts()

no       20247
yes      15262
nan       2176
twin       125
turbo        3
Name: Turbo Charger, dtype: int64

Well, cleaning this column should be quite straightforward.
1. `twin` and `turbo` should be replaced by `yes`

In [20]:
# Replace `twin` and `turbo` with `yes`
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('twin', 'yes')
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('turbo', 'yes')

cars_df['Turbo Charger'].value_counts()

no     20247
yes    15390
nan     2176
Name: Turbo Charger, dtype: int64

--- 

##### `Gear Box`

In [21]:
cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed                  22840
6 speed                   5876
5-speed                   1457
7 speed                   1256
6-speed                   1199
                         ...  
six speed manual             1
7g-dct                       1
amg speedshift dct 8g        1
amg 7-speed dct              1
ecvt                         1
Name: Gear Box, Length: 106, dtype: int64

Here the cardinality is high. And as is evident, several categories can be lumped together into a single one. Let's define a mapping to clearly indeicate which data should be lumped together.

In [22]:
gear_box_mapping = {}
gear_box_mapping['1 speed'] = [
	'single speed', 
	'single speed automatic', 
	'single speed reduction gear', 
	'single-speed transmission', 
]
gear_box_mapping['4 speed'] = [
	'4 speed', 
	'4-speed', 
]
gear_box_mapping['5 speed'] = [
	'5', 
	'5 - speed', 
	'5 gears', 
	'5 manual', 
	'5 speed', 
	'5 speed at+ paddle shifters', 
	'5 speed cvt', 
	'5 speed forward, 1 reverse', 
	'5 speed manual', 
	'5 speed manual transmission', 
	'5 speed+1(r)', 
	'5 speed,5 forward, 1 reverse', 
	'5-speed', 
	'5-speed`', 
	'five speed', 
	'five speed manual', 
	'five speed manual transmission', 
	'five speed manual transmission gearbox', 
]
gear_box_mapping['6 speed'] = [
	'6', 
	'6 speed', 
	'6 speed at', 
	'6 speed automatic', 
	'6 speed geartronic', 
	'6 speed imt', 
	'6 speed ivt', 
	'6 speed mt', 
	'6 speed with sequential shift', 
	'6-speed', 
	'6-speed at', 
	'6-speed automatic', 
	'6-speed autoshift', 
	'6-speed cvt', 
	'6-speed dct', 
	'6-speed imt', 
	'6-speed ivt', 
	'6-speed`', 
	'six speed  gearbox', 
	'six speed automatic gearbox', 
	'six speed automatic transmission', 
	'six speed geartronic, six speed automati', 
	'six speed manual', 
	'six speed manual transmission', 
	'six speed manual with paddle shifter', 
]
gear_box_mapping['7 speed'] = [
	'7 speed', 
	'7 speed 7g-dct', 
	'7 speed 9g-tronic automatic', 
	'7 speed cvt', 
	'7 speed dct', 
	'7 speed dsg', 
	'7 speed dual clutch transmission', 
	'7 speed s tronic', 
	'7-speed', 
	'7-speed dct', 
	'7-speed dsg', 
	'7-speed pdk', 
	'7-speed s tronic', 
	'7-speed s-tronic', 
	'7-speed steptronic', 
	'7-speed stronic', 
	'7g dct 7-speed dual clutch transmission', 
	'7g-dct', 
	'7g-tronic automatic transmission',
	'amg 7-speed dct',	
	'mercedes benz 7 speed automatic',
]
gear_box_mapping['8 speed'] = [
	'8', 
	'8 speed', 
	'8 speed cvt', 
	'8 speed multitronic', 
	'8 speed sport', 
	'8 speed tip tronic s', 
	'8 speed tiptronic', 
	'8-speed', 
	'8-speed automatic', 
	'8-speed automatic transmission', 
	'8-speed dct', 
	'8-speed steptronic', 
	'8-speed steptronic sport automatic transmission', 
	'8-speed tiptronic', 
	'8speed',
	'amg speedshift dct 8g', 
]
gear_box_mapping['9 speed'] = [
	'9 -speed', 
	'9 speed', 
	'9 speed tronic', 
	'9-speed', 
	'9-speed automatic', 
	'9g tronic', 
	'9g-tronic', 
	'9g-tronic automatic', 
	'amg speedshift 9g tct automatic',
]
gear_box_mapping['10 speed'] = [
	'10 speed', 
]
gear_box_mapping['cvt'] = [
'cvt', 
'e-cvt', 
'ecvt', 
]
gear_box_mapping['direct drive'] = [
'direct drive', 
]
gear_box_mapping['fully automatic'] = [
	'automatic transmission', 
	'fully automatic',
]
gear_box_mapping['nan'] = [
'nan',
'ags', 
'imt', 
'ivt', 
]

mapping_dict = {v: k for k, lst in gear_box_mapping.items() for v in lst}
cars_df['Gear Box'] = cars_df['Gear Box'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed            25203
6 speed             7459
7 speed             1842
8 speed             1150
4 speed              760
cvt                  474
nan                  471
9 speed              399
fully automatic       20
1 speed               15
direct drive          11
10 speed               9
Name: Gear Box, dtype: int64

--- 

##### `Drive Type`

In [23]:
cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd                                         27456
nan                                          4496
rwd                                          2248
awd                                          1082
2wd                                           648
4wd                                           570
2 wd                                          369
4x2                                           297
4x4                                           229
front wheel drive                             176
two wheel drive                                98
all wheel drive                                32
rear wheel drive with esp                      29
two whhel drive                                26
permanent all-wheel drive quattro              21
rwd(with mtt)                                  14
rear-wheel drive with esp                       7
4 wd                                            7
all-wheel drive with electronic traction        5
four whell drive                                2


We will once again take the same approach and define a mapping since many categories are redundant here.

In [24]:
drive_type_mapping = {}
drive_type_mapping['fwd'] = ['fwd', 'front wheel drive']
drive_type_mapping['2wd'] = ['2wd', 'two wheel drive', '2 wd', 'two whhel drive']
drive_type_mapping['rwd'] = ['rwd', 'rear wheel drive with esp', 'rear-wheel drive with esp', 'rwd(with mtt)']
drive_type_mapping['awd'] = ['awd', 'all wheel drive', 'all-wheel drive with electronic traction', 'permanent all-wheel drive quattro']
drive_type_mapping['4wd'] = ['4wd', '4 wd', '4x4', 'four whell drive']
drive_type_mapping['nan'] = ['nan', '3']

mapping_dict = {v: k for k, lst in drive_type_mapping.items() for v in lst}
cars_df['Drive Type'] = cars_df['Drive Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd    27632
nan     4497
rwd     2298
2wd     1141
awd     1140
4wd      808
4x2      297
Name: Drive Type, dtype: int64

---

##### `Steering Type`

In [25]:
# Steering Type
cars_df[columns_unexpected_cardinality[4]].value_counts()

power         31920
electric       3895
nan             808
manual          652
electronic      344
electrical      138
epas             49
hydraulic         5
mt                1
motor             1
Name: Steering Type, dtype: int64

There are 2 kinds of steering types (broadly) - 
1. Power (electric)
2. Manual (hydraulic, etc.)

In [26]:
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electrical', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electric', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electronic', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('epas', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('mt', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('motor', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')

cars_df[columns_unexpected_cardinality[4]].value_counts()

power     36348
nan         808
manual      657
Name: Steering Type, dtype: int64

---

##### `Front Brake Type` and `Rear Brake Type`

In [27]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc                                        22642
ventilated disc                             13178
ventilated discs                              400
solid disc                                    398
nan                                           327
disc & caliper type                           296
disk                                          205
ventilated disk                                78
drum                                           67
multilateral disc                              42
vantilated disc                                39
disc, 236 mm                                   22
ventlated disc                                 17
vacuum assisted hydraulic dual circuit w       12
ventillated disc                               11
disc & drum                                    10
disc,internally ventilated                      9
electric parking brake                          8
264mm ventilated discs                          8
caliper ventilated disc                         6


In [28]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum                                        29260
disc                                         5272
ventilated disc                               757
solid disc                                    612
nan                                           326
self-adjusting drum                           322
discs                                         301
disc & caliper type                           296
ventilated discs                              178
leading-trailing drum                         134
leading & trailing drum                        75
ventilated drum                                46
drums                                          40
self adjusting drum                            33
disc & drum                                    32
self adjusting drums                           31
drums 180 mm                                   22
drum in discs                                  17
vacuum assisted hydraulic dual circuit w       12
262mm disc & drum combination                   8


In [29]:
brake_type_mapping = {}
brake_type_mapping['disc'] = [
	'disc',
	'260mm discs',
	'disc brakes',
	'disc, 236 mm',
	'discs',
	'disk',
	'multilateral disc',
	'solid disc',
	'electric parking brake',
	'abs',
]
brake_type_mapping['ventilated disc'] = [
	'264mm ventilated discs',
	'booster assisted ventilated disc',
	'caliper ventilated disc',
	'disc brakes with inner cooling',
	'disc,internally ventilated',
	'vantilated disc',
	'ventilated & grooved steel discs',
	'ventilated disc',
	'ventilated disc with twin pot caliper',
	'ventilated discs',
	'ventilated disk',
	'ventillated disc',
	'ventillated discs',
	'ventlated disc',
	'ventilated drum in discs',
	'ventialte disc',
	'ventialted disc',
]
brake_type_mapping['carbon ceramic'] = [
	'carbon ceramic brakes',
	'carbon ceramic brakes.',
]
brake_type_mapping['disc & drum'] = [
	'disc & drum',
	'228.6 mm dia, drums on rear wheels',
	'262mm disc & drum combination',
	'drum in disc',
	'drum in discs',
]
brake_type_mapping['drum'] = [
	'drum',
	'203mm drums',
	'drum`',
	'drums',
	'drums 180 mm',
	'booster assisted drum',
	'drum brakes',
	'leading & trailing drum',
	'leading-trailing drum',
	'self adjusting drum',
	'self adjusting drums',
	'self-adjusting drum',
	'single piston sliding fist',
	'ventilated drum',
	'tandem master cylinder with servo assist',

]
brake_type_mapping['caliper'] = [
	'six piston claipers',
	'twin piston sliding fist caliper',
	'vacuum assisted hydraulic dual circuit w',
	'four piston calipers',
	'disc & caliper type',
]

In [30]:
mapping_dict = {v: k for k, lst in brake_type_mapping.items() for v in lst}
cars_df['Front Brake Type'] = cars_df['Front Brake Type'].replace(mapping_dict)
cars_df['Rear Brake Type'] = cars_df['Rear Brake Type'].replace(mapping_dict)

In [31]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc               23334
ventilated disc    13759
nan                  327
caliper              312
drum                  70
disc & drum           10
carbon ceramic         1
Name: Front Brake Type, dtype: int64

In [32]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum               29973
disc                6195
ventilated disc      944
nan                  326
caliper              310
disc & drum           64
carbon ceramic         1
Name: Rear Brake Type, dtype: int64

---

##### `Tyre Type`

In [33]:
cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless,radial               21770
tubeless                       5723
tubeless, radial               4994
tubeless tyres                 2408
radial                          606
radial, tubeless                389
tubeless radial tyres           301
radial, tubless                 296
nan                             257
tubeless tyres, radial          224
tubeless,radials                212
radial,tubeless                 158
tubless, radial                  92
runflat                          69
tubeless tyre                    50
tubeless, runflat                48
run-flat                         47
runflat tyres                    41
radial tubeless                  33
tubeless,runflat                 31
tubeless radial                  17
runflat tyre                     16
tubeless, radials                 7
radial with tube                  4
tubeless. runflat                 3
tubeless radial tyrees            3
tubeless tyres mud terrain        3
radial tyres                

In [34]:
tyre_type_mapping = {}
tyre_type_mapping['tubeless'] = [
	'tubeless tyres',
	'tubeless',
	'tubeless tyres mud terrain',
	'tubeless tyre',
]
tyre_type_mapping['tubeless radial'] = [
	'tubeless, radial',
	'tubeless,radial',
	'tubeless tyres, radial',
	'tubeless radial tyres',
	'radial, tubeless',
	'radial',
	'tubless, radial',
	'radial tubeless',
	'tubeless radial',
	'tubeless,radials',
	'tubeless radials',
	'radial,tubeless',
	'tubeless radial tyre',
	'radial, tubless',
	'tubless radial tyrees',
	'tubeless , radial',
	'tubeless, radials',
	'radial tyres',
]
tyre_type_mapping['runflat'] = [
	'runflat tyres',
	'runflat',
	'tubeless,runflat',
	'run-flat',
	'runflat tyre',
	'tubeless, runflat',
	'tubeless. runflat',
	'tubeless.runflat',
	'tubeless radial tyrees',
]
tyre_type_mapping['tube'] = [
	'radial with tube',
]

In [35]:
mapping_dict = {v: k for k, lst in tyre_type_mapping.items() for v in lst}
cars_df['Tyre Type'] = cars_df['Tyre Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless radial    29108
tubeless            8184
runflat              260
nan                  257
tube                   4
Name: Tyre Type, dtype: int64

---

##### `Fuel Injection System`

Although we expect a high cardinality from this feature, it's still worth a look if we could reduce the number of catgories by losing some of the variance.

(This step is optional and can be skipped. Also the mapping that is done is approximate and to the best of my knowledge, your mapping could differ)

In [36]:
cars_df['Fuel Suppy System'].value_counts()

mpfi                                      13443
crdi                                       9165
nan                                        5502
direct injection                           3831
pgm-fi                                      882
                                          ...  
intake port(multi-point)                      1
mfi                                           1
distributor-type diesel fuel injection        1
tfsi                                          1
pgm-fi (programmed fuel inject                1
Name: Fuel Suppy System, Length: 71, dtype: int64

In [37]:
fuel_injection_mapping = {
    "Gasoline Port Injection": [
    	"intelligent-gas port injection", 
    	"i-gpi",
    	"dohc",
    	"pfi"
	],
    "Multi-Point Fuel Injection": [
    	"mpfi", 
    	"multi-point injection", 
    	"mpfi+lpg", 
    	"mpfi+cng", 
    	"multipoint injection", 
    	"smpi", 
    	"mpi",
    	"multi point fuel injection",
    	"dpfi",
    	"mfi",
    	"multi point injection",
    	"msds",
    	"cng"
    ],
    "Electronic Fuel Injection": [
	    "efi(electronic fuel injection)", 
	    "efi", 
	    "efi (electronic fuel injection)", 
	    "efic", 
	    "electronic fuel injection", 
	    "electronically controlled injection", 
	    "electronic injection system", 
	    "sefi",
	    "egis",
	    "efi (electronic fuel injection",
	    "efi",
        "efi -electronic fuel injection",
    ],
    "Direct Injection": [
    	"direct injection", 
    	"direct injectio", 
    	"direct fuel injection",
    	"direct engine",
    ],
    "Common Rail Injection": [
    	"crdi", 
    	"common rail", 
    	"common rail injection", 
    	"common rail direct injection", 
    	"common rail direct injection (dci)", 
    	"common-rail type", 
    	"advanced common rail", 
    	"common rail system", 
    	"common rail diesel", 
    	"pgm-fi (programmed fuel injection)", 
    	"pgm-fi (programmed fuel inje", 
    	"pgm - fi", 
    	"pgm-fi", 
    	"pgm-fi (programmed fuel inject",
    	"direct injection common rail",
    	"cdi"
    ],
    "Distributor-Type Fuel Injection": [
    	"dedst", 
    	"distribution type fuel injection", 
    	"distributor-type diesel fuel injection",
    ],
    "Indirect Injection": [
    	"indirect injection",
    	"idi"
    ],
    "Gasoline Direct Injection": [
    	"gdi", 
    	"gasoline direct injection",
    	"tfsi",
    	"tsi",
    	"tgdi"
    ],
    "Turbo Intercooled Diesel": [
    	"tcdi", 
    	"turbo intercooled diesel",
    	"tdci"
    ],
    "Intake Port Injection": [
    	"intake port(multi-point)"
    ],
    "Diesel Direct Injection": [
    	"ddi", 
    	"ddis"
    ],
    "Variable Valve Timing Injection": [
    	"dual vvt-i", 
    	"vvt-ie", 
    	"ti-vct"
    ],
    "Three-Phase AC Induction Motors": [
    	"3 phase ac induction motors"
    ],
    "Electric": [
    	"electric", 
    	"isg"
    ],
}

mapping_dict = {v: k for k, lst in fuel_injection_mapping.items() for v in lst}
cars_df['Fuel Suppy System'] = cars_df['Fuel Suppy System'].replace(mapping_dict)

cars_df['Fuel Suppy System'].value_counts()

Multi-Point Fuel Injection         14055
Common Rail Injection              11822
nan                                 5502
Direct Injection                    3837
Electronic Fuel Injection           1593
Gasoline Direct Injection            653
Gasoline Port Injection              100
Diesel Direct Injection               99
Turbo Intercooled Diesel              89
Indirect Injection                    21
Variable Valve Timing Injection       15
Three-Phase AC Induction Motors       11
Distributor-Type Fuel Injection        8
Electric                               7
Intake Port Injection                  1
Name: Fuel Suppy System, dtype: int64

##### With this column, all of the features suspected of high cardinality are handled. We can move on to the other things that were pointed out earlier in the code.

---

#### Converting features to numerical (and/or boolean)

The features we wish to convert are the following:
1. `pu` to float
2. `Max Power` to 2 separate columns: `Max Power Delivered` and `Max Power At`
3. ~~`No of Cylinder` to int (from float)~~
4. ~~`Values per Cylinder` to int (from float)~~
5. `BoreX Stroke` to 2 separate columns: `Bore` and `Stroke`
6. `Turbo Charger` to boolean
7. `Super Charger` to boolean
8. `Length` to int 
9. `Width` to int
10. `Height` to int
11. `Wheel Base` to int
12. `Front Tread` to int
13. `Rear Tread` to int
14. `Kerb Weight` to int
15. `Gross Weight` to int
16. ~~`Seating Capacity` to int (from float)~~
17. `Turning Radius` to float
18. `Top Speed` to int
19. `Acceleration` to float
20. ~~`No Door Numbers` to int (from float)~~
21. `Cargo Volumn` to int
22. `Compression Ratio` to float
23. `Alloy Wheel Size` to float
24. `Ground Clearance Unladen` to int

In [38]:
# Make a copy of the data, cars_df will now serve as a checkpoint
cars_df2 = cars_df.copy()

# Sanity check
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,5.000,180-liters,used,uttar pradesh,silver,first,nan,NaN,nan,nan
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,5.000,235-litres,used,maharashtra,grey,first,Gasoline Port Injection,11.0:1,nan,nan
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,4.000,400-litres,used,delhi,silver,second,nan,NaN,nan,nan
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,4.000,nan,used,delhi,silver,second,Multi-Point Fuel Injection,NaN,13,nan
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,5.000,nan,used,maharashtra,white,first,nan,12.0+-.03,nan,nan


In [39]:
# A utility function to get a number from a string
def get_begin_number(x):
    x = str(x)
    reg_match = re.search(r'\d+', x)
    if reg_match is None:
        return None
    return int(reg_match.group())

def get_begin_float(x):
    x = str(x)
    reg_match = re.search(r'\d+\.?\d+', x)
    if reg_match is None:
        return None
    return float(reg_match.group())

---

#### `pu`

This is our target column (the price of our used cars). This should be a float or int.

In [40]:
cars_df2['pu'].sample(5)

usedCarSkuId
389944e4-f8db-44dc-8560-321c58a23e5e    25,00,000
e553e17b-ddd4-4171-af9c-9b502f40808f     5,00,000
e4d8c912-12c6-4bcb-a5f5-1a4e113dd264    12,00,000
848fda38-a1c3-41f7-abdf-4eb291c4775f     6,00,000
948e4a5e-218a-4133-9128-a75fda9e30b6     7,50,000
Name: pu, dtype: object

In [41]:
# We simply need to replace the ',' to with '' and convert
cars_df2['pu'] = cars_df2['pu'].str.replace(',', '').astype(float)

cars_df2['pu'].sample(5)

usedCarSkuId
be89daa8-ea9a-4756-81e5-1b0411ee8b7a    197953.000
c9c8c9b6-e38c-430f-8514-6dc265946dbf    220000.000
285c9e78-537c-4db7-afe2-71901e4aa58f    275000.000
ab39a330-a0c4-4927-9d2f-6c94ef24ddd9    301002.000
a86c2071-9826-405c-b595-6be540daa774   1050000.000
Name: pu, dtype: float64

---

#### `Max Power` and `Max Torque`

We will be separating this column into 2 columns: `Max Power Delivered` and `Max Power At` (containing the rpm at which the max power is reach)
Same with `Max Torque`

In [42]:
cars_df2[['Max Power', 'Max Torque']].sample(5)

,Max Power,Max Torque
usedCarSkuId,,
12309669-2a55-4ae4-9352-4a72fa472758,67.1bhp@6200rpm,90nm@3500rpm
030d8a97-1bd2-472e-bcdb-14d31c8669fe,nan,nan
10cbb6a5-d8ca-41e0-9aa2-a705a72bb005,88.7bhp@6000rpm,110nm@4800rpm
a20f8563-4f39-4b2b-8983-f6e6a169fe58,103.5bhp@5250rpm,153nm@3800rpm
a1ff7028-2355-4a91-a4ad-d2a3c6e825f0,138.1bhp@3750rpm,350nm@1750-2500rpm


In [43]:
c = cars_df2.copy()
cars_df2['Max Power Delivered'] = cars_df2['Max Power'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Power At'] = cars_df2['Max Power'].str.split('@').str[1].apply(get_begin_float).astype(float)


def get_rpm_average(x):
    x = str(x)
    if '-' in x:
        p1 = get_begin_float(x.split('-')[0])
        p2 = get_begin_float(x.split('-')[1])
        if p1 is None:
            return p2
        if p2 is None:
            return p1
        
        return (p1 + p2)/2
    else:
        return get_begin_float(x)

cars_df2['Max Torque Delivered'] = cars_df2['Max Torque'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Torque At'] = cars_df2['Max Torque'].str.split('@').str[1].apply(get_rpm_average).astype(float)

In [44]:
cars_df2[['Max Power Delivered', 'Max Power At', 'Max Torque Delivered', 'Max Torque At']].sample(5)

,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,
c128376d-4ebc-48e7-a5ab-fa11b25efe8d,83.100,6000.000,115.000,4000.000
2e7696fa-883e-4fdf-82b8-96e314bcc50f,85.800,6000.000,114.000,4000.000
21ef7c84-4b4a-476f-b1b9-34b81099acf6,112.000,5000.000,207.000,2200.000
97473ec8-3941-4ede-a9ca-35998b34315b,140.800,3500.000,350.000,5000.000
0eaf5b83-3dc7-4c46-afc3-0823a2cff282,74.000,4000.000,190.000,2000.000


Now we can safely drop `Max Power` from our dataframe

In [45]:
cars_df2.drop(columns=['Max Power'], inplace=True, axis=1)
cars_df2.drop(columns=['Max Torque'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,silver,first,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,grey,first,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,silver,second,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,silver,second,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,white,first,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000


---

#### `BoreX Stroke`

We will make 2 new columns `Bore` and `Stroke` and remove the original column.


In [46]:
cars_df2['BoreX Stroke'].sample(10)

usedCarSkuId
bae8cb26-442e-4aa2-9fa7-2d4d5e19a9d1               nan
c67da3ca-3716-4ecb-8286-d2ff9fa1ae2a           76x82.5
036f1484-7f18-44ec-b855-96386db014c0               nan
b23d695b-9bd9-47f8-b325-0e2b05ac2a91    68.5 x 72.0 mm
17e61aff-27b5-409e-8dc1-5f1151056c74               nan
84c8ba23-97fa-41a5-939b-6eed04b98007    73.5 x 88.3 mm
db880997-eb04-4afa-bb0c-cf4ae79e0999         83x92.4mm
0342e341-9b57-40fe-8678-23689d932ee4               nan
e5fc6069-74ab-4590-9f52-264ea3f4fc0a    68.5 x 67.5 mm
589a6111-ffe3-425f-bdd9-9b37d6021d67               nan
Name: BoreX Stroke, dtype: object

In [47]:
cars_df2['Bore'] = cars_df2['BoreX Stroke'].str.split('x').str[0].apply(get_begin_float).astype(float)
cars_df2['Stroke'] = cars_df2['BoreX Stroke'].str.split('x').str[1].apply(get_begin_float).astype(float)

In [48]:
cars_df2[['Bore', 'Stroke']].sample(10)

,Bore,Stroke
usedCarSkuId,,
bb3937d0-1099-4ca3-8884-df5ef16b6698,NaN,NaN
63bb09ad-63fa-4795-b8ea-3a697ba83437,NaN,NaN
a94d3ad7-7ca9-4a90-a216-66c314f4dddc,NaN,NaN
3bfbdf50-f411-493d-9703-dbb2bc084857,NaN,NaN
19029ae4-8c53-45db-97ca-088d40188470,NaN,NaN
51cd0c7f-0d19-46a4-9600-796a5b499364,NaN,NaN
0c2befa7-56d6-44d5-884e-02317152e1dc,74.000,75.500
e5d5ccee-981e-463b-b1c4-6d6d9d28c7cd,NaN,NaN
0ee07c87-d761-49d8-9146-bbce5607cda0,NaN,NaN


In [49]:
cars_df2.drop(columns=['BoreX Stroke'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000,69.000,72.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000,73.000,82.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000,NaN,NaN


---

#### `Turbo Charger` and `Super Charger`

Convert them to boolean

In [50]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
630780fb-f466-46e0-aef5-b88bab06abb9,no,no
44c60d34-31b4-437a-a233-78980a2d24eb,no,no
d87d294b-9124-4690-8ca9-8775889a2eca,yes,nan
988ca9d0-3e16-42fd-9b13-76c68d2b34f0,no,no
bacd5c27-76a4-46c4-8463-e1934c17af6a,no,no


In [51]:
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

In [52]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
f2587189-7e0e-44e9-80ba-bba2d63f5ead,True,True
4ebc071b-5522-4d76-b489-6c95fa6b0425,True,True
34784a55-7722-4d91-a5ab-246a65fbef73,True,True
b34078e0-da7f-49f3-a789-a2ba3fa26f20,True,True
45f74a67-adb1-4a3a-bb88-9b07f4d5ca09,True,True


---

#### `Length`, `Width`, `Height`, `Wheel Base`

Convert these columns to int

In [53]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(10)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
8bfbff00-e1d5-48f3-8d84-3c6ee7f9b1df,3840mm,1735mm,1530mm,2450mm
b8eb131b-9442-4027-a8cd-311cf2e3f0a6,3995mm,1790mm,1640mm,2500mm
ff64d8aa-863a-49f5-9b1f-7499be4bff5c,3954mm,1737mm,1570mm,2490mm
2f495e79-e362-48dc-aab5-b7c294dd5af3,3840mm,1735mm,1530mm,2450mm
62e2c3be-f48e-486e-892f-a560b50466db,4580mm,1770mm,1755mm,2750mm
176bc1e6-b2e7-4040-b0ef-fe13c7d2f4d7,4395mm,1818mm,1640mm,2636mm
2a05a79e-6864-489f-a302-0ef3183da56e,3990mm,1680mm,1505mm,2405mm
505b7b12-a900-4df0-a680-67afe184a1cc,4585mm,1890mm,1785mm,2700mm
dd6e52b5-d473-48d2-88f2-a73c33c23c59,3746mm,1647mm,1535mm,2400mm


In [54]:
for col in ['Length', 'Width', 'Height', 'Wheel Base']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [55]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(5)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
8c22dd9c-1b82-4f70-9337-1c2f828e0889,4250.000,1670.000,1370.000,2440.000
7a13a504-519c-48c4-a2d0-dbdff80451a0,4315.000,1822.000,1695.000,2673.000
a915e2f4-dfa4-44b5-a0a0-85d0eb3dea3b,3585.000,1595.000,1550.000,2380.000
d46f4dbd-a97c-4a81-a7aa-4e2fbe27e2da,4420.000,1695.000,1480.000,2550.000
f54c8b6a-4d5c-4dbb-b5d7-f7ef3c4b67c4,NaN,NaN,NaN,NaN


---

#### `Front Tread` and `Rear Tread`

In [56]:
cars_df2[['Front Tread', 'Rear Tread']].sample(10)

,Front Tread,Rear Tread
usedCarSkuId,,
18c9d17c-5a9f-4326-9b6b-d3f6accfaca9,nan,nan
16b312f7-0598-4701-b08b-824d7f7518a4,1440mm,1445mm
dd64cd4e-6858-4f34-a37c-86d116df50a2,1479mm,1493mm
2ce1a862-2025-4d32-89b1-7df9c220cd26,nan,nan
5a93b82d-268c-4872-97e0-a64bc68fa24e,nan,nan
770ebf5a-92ad-4857-8a73-0d616f566283,nan,nan
9eedd9d8-9563-4756-9ba3-866d6e9f86ad,nan,nan
5cefabb3-7738-4d77-a38a-82d321ae0f6b,1295mm,1290mm
0b3dfc91-b475-40fd-bb0d-d57e9e52c91e,1480mm,1465mm


In [57]:
for col in ['Front Tread', 'Rear Tread']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [58]:
cars_df2[['Front Tread', 'Rear Tread']].sample(5)

,Front Tread,Rear Tread
usedCarSkuId,,
3e785efa-ea96-405b-8d6d-d1c3dbb3ba4d,NaN,NaN
23b68134-394e-4c50-94d1-88fe17f61bbf,NaN,NaN
1afaedb6-f6ce-4bed-9d8b-41727d2aaab0,NaN,NaN
2e439dd3-c399-4d79-b034-04d661fb71e5,1295.000,1290.000
d9ce3e39-ae83-4c99-9dc3-283021507148,1295.000,1290.000


---

#### `Kerb Weight` and `Gross Weight` 

Convert weights to numerical features

In [59]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
fa7586df-236b-434a-a641-7d05b12c6a6a,1800kg,2280kg
c28bc36d-c6c8-4826-8c06-a85e5dbee5e3,1871kg,2400kg
6bdcfa78-a2a6-450d-b1c7-ebeaa0246f23,950kg,nan
d27750ea-cd0a-46e9-8207-6ef91d58e541,1066kg,nan
666dac52-5820-4891-8fe7-183455335610,1562 kgs,nan
6b7234f1-a299-4ad2-951f-0482560fbd50,1265kg,nan
437e3fe0-8985-4975-925e-c4cc1aaa9730,835kg,1250kg
66ce41d9-1141-441f-ae33-53bcc097869c,1066kg,nan
7c0d6afb-88ad-4857-895a-5d30059c5f82,nan,nan


In [60]:
for col in ['Kerb Weight', 'Gross Weight']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [61]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
ec2e7ca4-2e4a-43f1-bd73-6c7d7a0e7e65,870.000,1350.000
4bfb0564-14ec-43ac-bbf4-75c747aca28d,NaN,2510.000
5dda3a04-addd-4204-9273-df31b1177ecd,NaN,NaN
f59a2be6-c64f-443e-87a4-7b2ad6dd15ad,1135.000,1595.000
91c92d7d-6a12-46d2-a3aa-a9a79c40643a,NaN,NaN
ecdc31e2-0093-411d-8e47-a8bfdce975e9,960.000,1350.000
0f2e11ee-fee7-4843-b701-8d87924f70d1,NaN,NaN
6220ab31-3ee0-4cf7-b386-cc26b92f9882,NaN,NaN
fd0ce6ba-ef37-4a32-b676-36874a31b6a0,1880.000,2450.000


---

#### `Turning Radius`,

In [62]:
cars_df2['Turning Radius'].sample(10)

usedCarSkuId
e9d6264e-e6cb-4392-8624-5f41ee7377fb          4.7m
53aa41dd-8a04-407c-b7cf-e731ee477841         4.6 m
0fc13c9f-38d7-45ba-b8d8-24db3009e1a6    5.1 meters
5891e057-b974-4294-a1aa-feaa4568325b    4.8 metres
c65de030-9417-46da-bc4c-917c9f1e9c0e    5.4 meters
09090215-bbfb-4ca7-9c3f-4352711f2de8           5.0
4b569b05-e93c-4388-bf88-acadd897a1d4    5.6 metres
193e5097-0993-47e6-9bb4-679d2454748f           nan
71b6867b-384e-4737-8b40-71bf6ed0ffe4    5.2 metres
8f272ff3-7f7c-4d38-a313-d5d8e7f94e28    4.8 metres
Name: Turning Radius, dtype: object

In [63]:
cars_df2['Turning Radius'] = cars_df2['Turning Radius'].apply(get_begin_float).astype(float)

In [64]:
cars_df2['Turning Radius'].sample(5)

usedCarSkuId
fe9bdd22-cd0d-46c6-8db6-c3d432e9e434   5.200
1b56f3ab-9df9-4344-8f9e-96d74d533a82   4.800
dc89c6d1-b043-4e6e-822e-c3598e253150   4.800
e9c7168b-e7c5-48c5-9e8c-f2f6ca9ec95e   5.300
0ff58306-b431-4751-9478-b607740c7c59   4.700
Name: Turning Radius, dtype: float64

---

#### `Top Speed` and `Acceleration`

In [65]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
0130731e-9fda-4846-9797-8d911e72ccd7,185 kmph,10 seconds
6ad6f5da-09d5-4de1-b126-95aad7f5910a,158km/hr,15.2 seconds
fdd1459c-3604-4570-93c8-1fcf40720eb7,172 kmph,12.36 seconds
fd00b075-c74f-4ebd-841e-d671496f795e,nan,nan
583bcc22-3b7a-446f-bb65-843b57bb5206,145 kmph,15.7 seconds
4c4842a1-aa8c-4ac5-a2e0-60d1de3f0849,216 kmph,9.35 seconds
22f0aca0-9e2a-4268-8333-f2c06175f5ee,nan,13.3 seconds
c2132ad2-8000-4c4b-9318-c399ff60832c,160km/hr,12.29 seconds
739163b8-5b76-4ee0-82c0-c6dcc42f1f23,nan,nan


In [66]:
cars_df2['Top Speed'] = cars_df2['Top Speed'].apply(get_begin_float).astype(float)
cars_df2['Acceleration'] = cars_df2['Acceleration'].apply(get_begin_float).astype(float)

In [67]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
4e4448ee-8f74-4d12-8f7f-f5db51d4d817,185.000,10.000
ac5c7025-0149-4c73-b662-c8387f44220a,NaN,NaN
fd1ed2ab-4476-4943-9aad-0a8050d44d40,160.000,14.500
4ebf53b7-8910-43c3-947c-4feb145cb9c7,NaN,18.600
a010c808-4598-49f3-8f81-c5e9cff4e3a2,NaN,NaN
164850ae-ec5a-458e-a81b-88a28bcc114e,172.000,12.360
b748a64a-1bbe-4520-84cf-35f15955cefc,195.680,8.930
1113df28-0187-4efe-bc16-98221e8893ee,NaN,NaN
9891db0c-015a-47de-9b39-c288cbafda14,NaN,NaN


---

#### `Cargo Volumn` and `Ground Clearance Unladen`

Convert these columns to int

In [68]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
ce435f9c-c569-4de4-b04b-5dd7f68c08dd,236-liters,nan
da5a6fdc-7f59-42f9-936f-3b25617ecb71,256-liters,nan
0a1092eb-9520-488c-a57e-03f6e8b3d54b,280-liters,nan
20af2ab9-85d6-4a55-ae4c-807c99f3dae0,408-litres,nan
d5640f57-c9fd-4781-838e-3354038a9125,460-litres,nan
c44050db-81cb-4456-a231-c742ed3d936a,520-litres,nan
174d6974-4743-4e15-987d-48e0cc93744c,465-litres,nan
da4357d0-302b-4e93-9e41-033c91044399,480-litre,nan
7ec8d8f1-35d8-4e36-96d0-1e80a27544bc,510-litres,nan


In [69]:
for col in ['Cargo Volumn', 'Ground Clearance Unladen']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [70]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
b2ab71c7-3b2e-4146-8203-4107ee957155,NaN,NaN
da268524-6986-4035-951d-6d9d2f96a538,177.000,NaN
39c945ae-b699-4d60-9ee8-5125718f2e26,540.000,NaN
5ac9ed8e-257f-472d-88bf-23e0f2818b49,506.000,NaN
1be342e4-0b5a-413d-9833-53dd75b8dbeb,300.000,NaN
811fe862-2cef-40b7-9709-764de8e6ab70,540.000,NaN
6c2d8c39-9365-4090-93a1-cae44505bedc,232.000,NaN
267094f0-07c2-4b2a-9a8a-794d1be5414b,155.000,192.000
8f39663b-9252-4bf2-893c-e68adabeb622,256.000,NaN


---

#### `Compression Ratio` and `Alloy Wheel Size`

These columns should be converted to floats

In [71]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
d3b37838-aab4-45c7-a7b2-1a8c3e55e563,15,NaN
e8b2882c-33c3-4321-8b3f-97e0381ecf1b,16,NaN
ab589494-1746-4a10-9473-c106505759e4,14,9.0:1
5abfe5d2-bed0-4b13-a83d-242926542cf9,16,10.3:1
7f6cf230-823b-4233-be19-798039e0df98,17,NaN
52520c6c-6de2-4cc3-88b1-9d352132d26b,13,NaN
477ece9c-7ae7-43f2-a971-60f7142a5267,nan,NaN
16eb3352-803d-4e48-9a46-ab7e55e341b2,16,NaN
1c9379aa-851b-47cd-a2a1-a1757fc853c8,nan,NaN


In [72]:
for col in ['Alloy Wheel Size', 'Compression Ratio']:
    cars_df2[col] = cars_df2[col].apply(get_begin_float).astype(float)

In [73]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
6900ad88-e5b9-44c6-973f-649c2a0a7e36,NaN,17.900
3a2b3ac4-e928-4155-bd53-c6faf0c09849,NaN,10.300
3432bc9d-f19c-46e4-9527-245acbdd76f0,19.000,17.700
6dcf0a4b-26b7-4a7a-83d9-928f7b1125b6,NaN,NaN
8fe26faf-86ce-4d7a-a869-1bb3a6c213f7,15.000,10.500
edb909c6-89d9-414e-ba4b-d19f685578a4,NaN,NaN
ddf5b9c9-58b1-46d8-83e8-3a8938616a43,14.000,NaN
941228e5-d6f2-473b-befc-d751163d3ee6,NaN,NaN
57f03c0f-0e13-4143-b80e-0d15abdc65e9,15.000,NaN


With this, we have converted all the columns to numerical that were noted.

In [74]:
cars_df2.dtypes.value_counts()

object     30
float64    26
int64       4
bool        3
dtype: int64

---

#### Fixing some column names

In [75]:
cars_df2 = cars_df2.rename(columns={
    'pu': 'listed_price',
    'Values per Cylinder': 'Valves per Cylinder',
    'Value Configuration': 'Valve Configuration',
    'No Door Numbers': 'Doors',
    'Seating Capacity': 'Seats',
    'Cargo Volumn': 'Cargo Volume',
    'city_x': 'City',
})

In [76]:
cars_df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti wagon r lxi cng,maruti,...,nan,NaN,NaN,NaN,58.160,6200.000,77.000,3500.000,69.000,72.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,maruti celerio green vxi,maruti,...,Gasoline Port Injection,11.000,NaN,NaN,58.200,6000.000,78.000,3500.000,73.000,82.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,honda amaze s plus i-vtec,honda,...,nan,NaN,NaN,NaN,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,0,[{'img': ''}],0,False,maruti wagon r lxi cng,maruti,...,Multi-Point Fuel Injection,NaN,13.000,NaN,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,maruti ertiga vxi cng,maruti,...,nan,12.000,NaN,NaN,86.630,5500.000,121.500,4200.000,NaN,NaN


In [77]:
cars_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37813 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loc                       37813 non-null  object 
 1   myear                     37813 non-null  int64  
 2   bt                        37813 non-null  object 
 3   tt                        37813 non-null  object 
 4   ip                        37813 non-null  int64  
 5   images                    37813 non-null  object 
 6   imgCount                  37813 non-null  int64  
 7   threesixty                37813 non-null  bool   
 8   dvn                       37813 non-null  object 
 9   oem                       37813 non-null  object 
 10  model                     37813 non-null  object 
 11  variantName               37813 non-null  object 
 12  City                      37813 non-null  object 
 13  

#### With this, we are ready to move on to the next steps:
1. EDA
2. Feature Engineering and selection
3. Model Building and Optimization

In [78]:
# fileName = f"../data/clean/cleaned_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
# cars_df2.to_csv('fileName')